In [2]:
import requests
import json

from functions import get_municipalities

In [38]:
municipalities = get_municipalities()

array = [{'id': i['id'], 'en': i['en']} for i in municipalities]  # Общий список из 309 муниципалитетов
mun10 = [array[i:i+10] for i in range(0, len(array), 10)]  # Список по 10 муниципалитетов

In [54]:
def order(email: str):
    # Куки не долговечны, если будете использовать повторно не забудьте обновить
    # https://curlconverter.com
    # В консоли разработчика ищем запрос - createPayment > копировать > копировать все как cURL (bash)

    cookies = {
        'BIGipServerkakssasitu01-karttapaikka-pool': '2944673034.36895.0000',
        'i18next': 'en',
        '_pk_ref.10.0801': '%5B%22%22%2C%22%22%2C1690998723%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D',
        '_pk_id.10.0801': '2317dc987081bc55.1690998723.',
        '_pk_ses.10.0801': '1',
        'XSRF-TOKEN': '9482ab5b-672b-4a2b-aca1-b690fd7c5443',
        'TS01ec89f6': '01b1a68b73e038f91a47b6dbf6caaaeb7515c366a026bfbd88b79ae115de5073129261c70e16cb3df4b5fc70ab238682f9e7c6c22fc8d95b4755f444d8640309e77bc3c16c45c5cef079f7a0c3246e1f0472abd105',
    }

    headers = {
        'Accept': 'application/json',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        # 'Cookie': 'BIGipServerkakssasitu01-karttapaikka-pool=2944673034.36895.0000; i18next=en; _pk_ref.10.0801=%5B%22%22%2C%22%22%2C1690998723%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_id.10.0801=2317dc987081bc55.1690998723.; _pk_ses.10.0801=1; XSRF-TOKEN=9482ab5b-672b-4a2b-aca1-b690fd7c5443; TS01ec89f6=01b1a68b73e038f91a47b6dbf6caaaeb7515c366a026bfbd88b79ae115de5073129261c70e16cb3df4b5fc70ab238682f9e7c6c22fc8d95b4755f444d8640309e77bc3c16c45c5cef079f7a0c3246e1f0472abd105',
        'Origin': 'https://asiointi.maanmittauslaitos.fi',
        'Referer': 'https://asiointi.maanmittauslaitos.fi/karttapaikka/kassa/2',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0',
        'X-XSRF-TOKEN': '9482ab5b-672b-4a2b-aca1-b690fd7c5443',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Opera GX";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    json_data = {
        'lang': 'en',
        'appContext': 'karttapaikka',
        'organizationId': None,
        'organizationName': None,
        'contactPerson': None,
        'firstName': '',
        'lastName': '',
        'phoneNumber': '',
        'email': f'{email}',
        'streetAddress': '',
        'postalCode': '',
        'city': '',
        'sessionId': '17bcc4c65d2630ded57f941924862676e7d5dae4869549e607443fac09c02578611430a90e1059fbae5eaa592a5e07c3a14ec251987b2f78bef7cf02b7ed5720',
        'purchaseRows': [],
        'spatialDataFilesOrders': [],
    }

    for list in mun10:
        for mun in list:
            json_data['spatialDataFilesOrders'].append(
                {
                    'productType': 'fileDownload',
                    'description': f'{mun["en"]} - GeoPackage\nBuildings',
                    'price': '0',
                    'unmodifiable': True,
                    'name': 'Topographic database',
                    'productId': 'maastotietokanta',
                    'areaSelectionType': 'kunta',
                    'polygon': None,
                    'bbox': None,
                    'freeSelectionType': None,
                    'fileFormat': 'GPKG',
                    'theme': 'rakennukset',
                    'mapSheets': None,
                    'municipality': f'{mun["id"]}',
                    'year': 9999,
                },
            )

    response = requests.post(
        'https://asiointi.maanmittauslaitos.fi/karttapaikka/api/createPayment',
        cookies=cookies,
        headers=headers,
        json=json_data,
    )

200

In [58]:
def get_my_purchases():
    response = requests.get('https://asiointi.maanmittauslaitos.fi/lataukset/api/spatialdatafilesorders/8aa5042f898eec620189b7911ab60923').json()['files']

    links = []

    for product in response:
        try:
            url = product['results']['results'][1]['path'] if product['results']['results'][0]['path'] is None else product['results']['results'][0]['path']
            current_product = {
                'name': product['inputs']['description'].split('\n')[0].replace(' - GeoPackage', ''),
                'url': url
                }
            links.append(current_product)
            print(current_product)
        except Exception as ex:
            print(f'Error: {ex}')

    json_purchases = json.dumps(links, indent=4)

    with open('data/json/purchases_link.json', 'w') as outfile:
        outfile.write(json_purchases)

    return links

In [59]:
links = get_my_purchases()

{'name': 'Pyhäranta', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/181e3de6-3d96-4199-96fe-4d892e726cc7/rakennukset_Pyhäranta.gpkg'}
{'name': 'Toholampi', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/d3560fdd-facf-4587-86ed-f583f3abf451/rakennukset_Toholampi.gpkg'}
{'name': 'Kyyjärvi', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/ff2e35f0-aa43-4f56-87d5-89c1b41706f4/rakennukset_Kyyjärvi.gpkg'}
{'name': 'Pudasjärvi', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/2bb34a90-3e5c-44f6-ae34-03785c89930f/rakennukset_Pudasjärvi.gpkg'}
{'name': 'Nakkila', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/c48065fd-e287-4599-88b7-19bf368c45b9/rakennukset_Nakkila.gpkg'}
{'name': 'Iitti', 'url': 'https://avoin-paikkatieto.maanmittauslaitos.fi/tiedostopalvelu/dl/v1/469bc807-f2b3-4121-909c-5cec4c1a81fb/rakennukset_Iitti.gpkg'}
{'name': 'Evijärvi', '

In [62]:
def download_my_products(my_purchases):
    print('Downloading files!')
    for (i, product) in enumerate(links):
        response = requests.get(url=product['url'])
        with open(f"data/raw/trash/{product['name']}.gpkg", 'wb') as file:
            file.write(response.content)
        print(f'{i}/{len(links)} File {product["name"]}.gpkg saved.')

0/308 File Pyhäranta.gpkg saved.
1/308 File Toholampi.gpkg saved.
2/308 File Kyyjärvi.gpkg saved.
3/308 File Pudasjärvi.gpkg saved.


KeyboardInterrupt: 